# Deploy best model from an AutoML Run

## Configure variables
- Only required to modify the following cell.
- After modification, run all the cells.
- Once all of them succeeds, you will have a successful deployment, endpoint name will be printed in the first cell's output.

In [ ]:
# Please enter the required information below

import datetime


# copy from UI at top right corner of the page (besides your profile picture)
subscription_id = ""
resource_group = ""
workspace_name = ""

# copy the "Name" of a successful run from UI
job_name = ""

model_name = "test-1"
model_description = ""

endpoint_name = "test-3-" + datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint_description = ""

deployment_name = "test-1"

print(f"Endpoint name: {endpoint_name}")

## Get handle to workspace

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import automl
from azure.ai.ml import Input


credential = DefaultAzureCredential()
ml_client = None
subscription_id = subscription_id
resource_group = resource_group
workspace = workspace_name

ml_client = MLClient(credential, subscription_id, resource_group, workspace)

## Get the AutoML best child run

In [ ]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

In [ ]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

In [ ]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

In [ ]:
# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

In [ ]:
# Get the desired child run

# specify the child run index, child runs are sorted by their normalized root mean squared error
# 0 means the best run, 1 means the second best run, and so on
child_run_index = 2

exp_id = mlflow_parent_run.info.experiment_id
parent_run_id = mlflow_parent_run.info.run_id
child_runs = mlflow_client.search_runs(
    experiment_ids=exp_id,
    filter_string=f"tags.mlflow.parentRunId = '{parent_run_id}' and attributes.status = 'Finished' and attributes.run_id != '{parent_run_id}_setup'",
)
child_runs = [cr for cr in child_runs if "normalized_root_mean_squared_error" in cr.data.metrics]
child_runs = sorted(child_runs, key=lambda cr: cr.data.metrics["normalized_root_mean_squared_error"])
best_run = child_runs[child_run_index]
best_run

In [ ]:
# # Get the best model's child run

# best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
# print("Found best child run id: ", best_child_run_id)

# best_run = mlflow_client.get_run(best_child_run_id)

# print("Best child run: ")
# print(best_run)

## Download the model

In [ ]:
# Create local folder
import os

local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [ ]:
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

## Register the model

In [ ]:
import datetime
from azure.ai.ml.entities import (
    Environment,
    Model,
)

model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/model.pkl",
    name=model_name,
    description=model_description,
)
registered_model = ml_client.models.create_or_update(model)

## Deployment

In [ ]:
# Creating a unique endpoint name with current datetime to avoid conflicts
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    CodeConfiguration,
    ProbeSettings,
    OnlineRequestSettings,
)


# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description=endpoint_description,
    auth_mode="key",
)

code_configuration = CodeConfiguration(
    code="artifact_downloads/outputs/", scoring_script="scoring_file_v_2_0_0.py"
)

In [ ]:
from azure.ai.ml.entities import OnlineRequestSettings

deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=registered_model.id,
    environment="azureml://registries/azureml/environments/AzureML-AutoML/versions/142",
    code_configuration=code_configuration,
    instance_type="Standard_DS3_V2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=180000
    )
)
# deployment to take 100% traffic
endpoint.traffic = {deployment_name: 100}
ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).wait()